In [1]:
# Import library
import os
import numpy as np
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [2]:
# Fungsi untuk meload gambar dari folder
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path):
            try:
                # Resize image agar seragam (misalnya 64x64)
                img = Image.open(file_path).resize((64, 64))
                img_array = np.array(img)  # Konversi ke array
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {file_path}: {e}")
    return images, labels

In [3]:
# Path dataset
dataset_path = "dataset/"
classes = ["cardboard", "metal", "paper", "plastic"]

# Load semua gambar dan label
X = []  # Untuk menyimpan gambar
y = []  # Untuk menyimpan label

for class_name in classes:
    folder_path = os.path.join(dataset_path, class_name)
    images, labels = load_images_from_folder(folder_path, class_name)
    X.extend(images)
    y.extend(labels)

# Convert ke NumPy array
X = np.array(X)
y = np.array(y)

Pre Prosesing

In [4]:
# Shuffle data agar acak
X, y = shuffle(X, y, random_state=42)

# Normalisasi gambar (skala 0-1)
X = X / 255.0

# Encode label menjadi angka
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [5]:
# Split data menjadi training dan validation set
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [7]:
# Reshape data ke bentuk 2D untuk Random Forest (karena RF tidak menerima array 3D)
n_samples, height, width, channels = X_train.shape
X_train_flat = X_train.reshape(n_samples, height * width * channels)

n_val_samples = X_val.shape[0]
X_val_flat = X_val.reshape(n_val_samples, height * width * channels)

Modeling

In [8]:
# Membuat dan melatih model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flat, y_train)

RandomForestClassifier(random_state=42)

Evaluation

In [9]:
# Evaluasi model
y_pred = rf_model.predict(X_val_flat)
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=label_encoder.classes_))

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.83      0.79      0.81        87
       metal       0.72      0.77      0.75        82
       paper       0.81      0.85      0.83       106
     plastic       0.84      0.79      0.81       103

    accuracy                           0.80       378
   macro avg       0.80      0.80      0.80       378
weighted avg       0.80      0.80      0.80       378



In [10]:
# Menguji pada folder trash
def predict_trash(folder, model, encoder):
    images, filenames = [], []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path).resize((64, 64))
                img_array = np.array(img) / 255.0
                images.append(img_array.flatten())  # Flatten untuk Random Forest
                filenames.append(filename)
            except Exception as e:
                print(f"Error loading image {file_path}: {e}")
    images = np.array(images)
    predictions = model.predict(images)
    predicted_labels = encoder.inverse_transform(predictions)
    return list(zip(filenames, predicted_labels))

# Path folder test/trash
test_path = "test/trash/"
trash_predictions = predict_trash(test_path, rf_model, label_encoder)
print("Predictions for trash folder:")
for file, label in trash_predictions:
    print(f"{file}: {label}")

Predictions for trash folder:
trash1.jpg: metal
trash10.jpg: metal
trash100.jpg: metal
trash101.jpg: metal
trash102.jpg: metal
trash103.jpg: metal
trash104.jpg: metal
trash105.jpg: metal
trash106.jpg: metal
trash107.jpg: metal
trash108.jpg: metal
trash109.jpg: paper
trash11.jpg: cardboard
trash110.jpg: plastic
trash111.jpg: plastic
trash112.jpg: metal
trash113.jpg: metal
trash114.jpg: metal
trash115.jpg: paper
trash116.jpg: paper
trash117.jpg: paper
trash118.jpg: paper
trash119.jpg: metal
trash12.jpg: metal
trash120.jpg: plastic
trash121.jpg: metal
trash122.jpg: plastic
trash123.jpg: cardboard
trash124.jpg: cardboard
trash125.jpg: cardboard
trash126.jpg: cardboard
trash127.jpg: cardboard
trash128.jpg: cardboard
trash129.jpg: cardboard
trash13.jpg: metal
trash130.jpg: cardboard
trash131.jpg: cardboard
trash132.jpg: paper
trash133.jpg: metal
trash134.jpg: cardboard
trash135.jpg: paper
trash136.jpg: paper
trash137.jpg: paper
trash14.jpg: cardboard
trash15.jpg: metal
trash16.jpg: metal
tra